# Week 7 Homework

***Due (pushed to your GitHub branch) on 11/1 by 11:59 pm***

## Samtools

Use Samtools to convert to BAM and sort the output SAMs produced by HISAT:

Merge the BAMs to a single BAM and generate an index:

Use Samtools `flagstat` to generate QC metrics for the STAR and HISAT merged BAMs:

## Qualimap

Use Qualimap to run `bamqc` and `rnaseq` to generate QC metrics for the STAR and HISAT merged BAMs:

## Report

Use MuliQC to generate a single report that merges all Samtools and Qualimap metrics:

## JBrowse2

Upload a screenshot of JBrowse2 that shows the reference genome, the annotations, and the two alignments as tracks.

## Snakemake Pipeline

1. Copy your `Snakefile` and `config.yaml` from `6_alignment` to `7_alignment_qc`. 
2. In the `Snakefile`, add the following rules:  
    a. `convert_hisat` - use `samtools` to convert HISAT SAMs to BAMs; sort the BAMs.
    b. `merge_hisat` - use `samtools` to merge all HISAT BAMs. 
    c. `flagstat` -  use `samtools` to generate `flagstat` QC. 
    d. `qualimap` - use `qualimap` to generate `bamqc` and `rnaseq` QC.
    e. `multiqc2` - use `multiqc` to aggregate all QC data into a single report.

This `Snakefile` should not use any `wrappers` because we already have the relevant software installed in the `biol343` conda environment. The pipeline should run to completion when any instructor or classmate runs `snakemake --use-conda`.